In [90]:
# Extract ERP for each year (Estimated Residential Population)
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

# Uncomment after downloading the raw csv file from ADS 2 Dataset folder on Google drive
'''

## Rental price --------------------------------------------
rental_history = spark.read.csv("../../data/raw/Rental price VIC SA2 2000 2021.csv", header=True)
convert2016to2021 = spark.read.csv("../../data/raw/external_dataset/sa2_2016_to_2021.csv", header=True)

# Victoria state data only
df = rental_history.filter(rental_history['state'] == 'VIC')

# Dateyear after 2010 only
df = df.withColumn("dateyear",df["dateyear"].cast(IntegerType()))
df = df[df['dateyear'] > 2012]

# Extract rental price related attributes only
df = df.select('dateyear', 'datemonth', 'sa22016name', 'sa22016code', 'propertycategorisation', 'for_rent_home_lease_averageprice', 'for_rent_home_lease_maximumprice', 'for_rent_home_lease_minimumprice')
df = df.sort(df["dateyear"].asc())


# Convert 2016 SA2 codes to 2022 SA2 codes
df = df.join(convert2016to2021, convert2016to2021.SA2_MAINCODE_2016 ==  df.sa22016code).\
    select(df.dateyear, df.datemonth, convert2016to2021.SA2_CODE_2021, convert2016to2021.SA2_NAME_2021, df.propertycategorisation, \
    df.for_rent_home_lease_averageprice, df.for_rent_home_lease_maximumprice, df.for_rent_home_lease_minimumprice).\
    sort("dateyear", 'datemonth')

# Write and save it 
df = df.na.drop()
df.write.csv("../../data/curated/rental_history_Vic.csv")

'''

## ERP (estimated Residential Population) for each SA2 Code--------------------------------------------------------
erp_all_state = spark.read.csv("../../data/raw/ABS_regional_population_sa2_2001_2021.csv", header=True)

# Victoria state data only
erp_vic = erp_all_state.filter(erp_all_state['state_name_2016'] == 'Victoria')

# Extract rental price related attributes only
erp_vic = erp_vic.select('sa2_name_2016', 'sa2_maincode_2016', 'erp_2013', 'erp_2014', 'erp_2015', 'erp_2016', 'erp_2017', 'erp_2018', 'erp_2019', 'erp_2020', 'erp_2021')

# Write and save it 
erp_vic = erp_vic.na.drop()
erp_vic.write.csv("../../data/curated/erp_vic.csv")

erp_vic.show()

+--------------------+-----------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|       sa2_name_2016|sa2_maincode_2016|erp_2013|erp_2014|erp_2015|erp_2016|erp_2017|erp_2018|erp_2019|erp_2020|erp_2021|
+--------------------+-----------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|       Gordon (Vic.)|        201021012|    5582|    5631|    5692|    5761|    5824|    5899|    5999|    6075|    6114|
|               Avoca|        201031013|    3337|    3322|    3317|    3323|    3330|    3352|    3364|    3370|    3409|
|            Beaufort|        201031014|    4047|    4172|    4317|    4454|    4485|    4450|    4547|    4615|    4637|
|Golden Plains - N...|        201031015|    4284|    4363|    4444|    4540|    4635|    4742|    4862|    4938|    5008|
|        Yackandandah|        204031075|    4489|    4511|    4536|    4578|    4586|    4627|    4683|    4756|    4826|
|              Drouin|  

In [84]:
# Make a new folder for features of interest dataset by year
import os
  
# Directory
directory = "features_of_interst"
  
# Parent Directory path
parent_dir = "../../data/curated/"

# Path
path = os.path.join(parent_dir, directory)

# Create the directory
os.mkdir(path)

# Create the directory for primary school, secondary school, park
parent_dir = "../../data/curated/features_of_interst/"
directory_list = ["primary", "secondary", "park", "train_station"]
for i in directory_list:
    directory = i
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)

In [82]:
import requests
import json
from csv import writer

# Primary school 
primary_2013 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272013-01-01%27+AND+%272013-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
primary_2014 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272014-01-01%27+AND+%272014-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
primary_2015 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272015-01-01%27+AND+%272015-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
primary_2016 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272016-01-01%27+AND+%272016-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
primary_2017 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272017-01-01%27+AND+%272017-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
primary_2018 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272018-01-01%27+AND+%272018-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
primary_2019 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272019-01-01%27+AND+%272019-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
primary_2020 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272020-01-01%27+AND+%272020-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
primary_2021 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272021-01-01%27+AND+%272021-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
primary_before = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27+AND+%28REGISTRATION_DATE+BETWEEN+%271900-01-01%27+AND+%272012-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='

primary_by_year_list = [primary_before, primary_2013, primary_2014, primary_2015, primary_2016, primary_2017, primary_2018, primary_2019, primary_2020, primary_2021]
primary_by_year_name = ['primary_before_2013', 'primary_2013', 'primary_2014', 'primary_2015', 'primary_2016', 'primary_2017', 'primary_2018', 'primary_2019', 'primary_2020', 'primary_2021']

# Secondary school 
secondary_2013 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272013-01-01%27+AND+%272013-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
secondary_2014 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272014-01-01%27+AND+%272014-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
secondary_2015 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272015-01-01%27+AND+%272015-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
secondary_2016 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272016-01-01%27+AND+%272016-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
secondary_2017 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272017-01-01%27+AND+%272017-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
secondary_2018 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272018-01-01%27+AND+%272018-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
secondary_2019 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272019-01-01%27+AND+%272019-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
secondary_2020 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272020-01-01%27+AND+%272020-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
secondary_2021 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%272021-01-01%27+AND+%272021-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
secondary_before = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27+AND+%28REGISTRATION_DATE+BETWEEN+%271900-01-01%27+AND+%272012-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='

secondary_by_year_list = [secondary_before, secondary_2013, secondary_2014, secondary_2015, secondary_2016, secondary_2017, secondary_2018, secondary_2019, secondary_2020, secondary_2021]
secondary_by_year_name = ['secondary_before_2013', 'secondary_2013', 'secondary_2014', 'secondary_2015', 'secondary_2016', 'secondary_2017', 'secondary_2018', 'secondary_2019', 'secondary_2020', 'secondary_2021']

# Park 
park_2013 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%272013-01-01%27+AND+%272013-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
park_2014 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%272014-01-01%27+AND+%272014-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
park_2015 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%272015-01-01%27+AND+%272015-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
park_2016 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%272016-01-01%27+AND+%272016-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
park_2017 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%272017-01-01%27+AND+%272017-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
park_2018 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%272018-01-01%27+AND+%272018-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
park_2019 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%272019-01-01%27+AND+%272019-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
park_2020 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%272020-01-01%27+AND+%272020-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
park_2021 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%272021-01-01%27+AND+%272021-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
park_before = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27+AND+%28REGISTRATION_DATE+BETWEEN+%271900-01-01%27+AND+%272012-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='

park_by_year_list = [park_before, park_2013, park_2014, park_2015, park_2016, park_2017, park_2018, park_2019, park_2020, park_2021]
park_by_year_name = ['park_before_2013', 'park_2013', 'park_2014', 'park_2015', 'park_2016', 'park_2017', 'park_2018', 'park_2019', 'park_2020', 'park_2021']

# Station 
station_2013 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%272013-01-01%27+AND+%272013-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
station_2014 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%272014-01-01%27+AND+%272014-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
station_2015 ='https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%272015-01-01%27+AND+%272015-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
station_2016 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%272016-01-01%27+AND+%272016-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
station_2017 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%272017-01-01%27+AND+%272017-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
station_2018 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%272018-01-01%27+AND+%272018-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
station_2019 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%272019-01-01%27+AND+%272019-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
station_2020 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%272020-01-01%27+AND+%272020-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
station_2021 = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%272021-01-01%27+AND+%272021-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='
station_before = 'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27RAST%27+AND+%28REGISTRATION_DATE+BETWEEN+%271900-01-01%27+AND+%272012-12-31%27+%29+&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelContains&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=standard&f=pgeojson&token='

station_by_year_list = [station_before, station_2013, station_2014, station_2015, station_2016, station_2017, station_2018, station_2019, station_2020, station_2021]
station_by_year_name = ['station_before_2013', 'station_2013', 'station_2014', 'station_2015', 'station_2016', 'station_2017', 'station_2018', 'station_2019', 'station_2020', 'station_2021']

# Column names
column_names = ["Place_Names", "Longitude", "Latitude"]

In [85]:
# Features of Interest by year and locations Dataset preprocessing 
for j in range(len(primary_by_year_list)):
    info_list = []
    print(j, primary_by_year_name[j])

    # Name the csv file
    csv_name =  '../../data/curated/features_of_interst/primary/' + str(primary_by_year_name[j]) + '.csv'

    for k in range(j):
        resp = requests.get(url=primary_by_year_list[k])

        content = json.loads(resp.text)

        # Get request for each property ID and extract attributes
        for i in content['features']:
            place_name = i['properties']['PLACE_NAME']
            longitude = i['geometry']['coordinates'][0]
            latitude = i['geometry']['coordinates'][1]

            info = [place_name, latitude, longitude]  
            info_list.append(info)

    print(len(info_list))

    with open(csv_name, 'a', newline='') as g:
        thewriter = writer(g)  
        thewriter.writerow(column_names)      
        thewriter.writerows(info_list)



0 primary_before_2013
0
1 primary_2013
1913
2 primary_2014
2489
3 primary_2015
2492
4 primary_2016
2494
5 primary_2017
2503
6 primary_2018
2507
7 primary_2019
2516
8 primary_2020
2527
9 primary_2021
2547


In [86]:
# Features of Interest by year and locations Dataset preprocessing 
for j in range(len(secondary_by_year_list)):
    info_list = []
    print(j, secondary_by_year_name[j])

    # Name the csv file
    csv_name =  '../../data/curated/features_of_interst/secondary/' + str(secondary_by_year_name[j]) + '.csv'

    for k in range(j):
        resp = requests.get(url=secondary_by_year_list[k])

        content = json.loads(resp.text)

        # Get request for each property ID and extract attributes
        for i in content['features']:
            place_name = i['properties']['PLACE_NAME']
            longitude = i['geometry']['coordinates'][0]
            latitude = i['geometry']['coordinates'][1]

            info = [place_name, latitude, longitude]  
            info_list.append(info)

    print(len(info_list))

    with open(csv_name, 'a', newline='') as g:
        thewriter = writer(g)      
        thewriter.writerow(column_names)   
        thewriter.writerows(info_list)

0 secondary_before_2013
0
1 secondary_2013
857
2 secondary_2014
1328
3 secondary_2015
1332
4 secondary_2016
1333
5 secondary_2017
1335
6 secondary_2018
1340
7 secondary_2019
1347
8 secondary_2020
1360
9 secondary_2021
1367


In [87]:
# Features of Interest by year and locations Dataset preprocessing 
for j in range(len(park_by_year_list)):
    info_list = []
    print(j, park_by_year_name[j])

    # Name the csv file
    csv_name =  '../../data/curated/features_of_interst/park/' + str(park_by_year_name[j]) + '.csv'

    for k in range(j):
        resp = requests.get(url=park_by_year_list[k])

        content = json.loads(resp.text)

        # Get request for each property ID and extract attributes
        for i in content['features']:
            place_name = i['properties']['PLACE_NAME']
            longitude = i['geometry']['coordinates'][0]
            latitude = i['geometry']['coordinates'][1]

            info = [place_name, latitude, longitude]  
            info_list.append(info)

    print(len(info_list))

    with open(csv_name, 'a', newline='') as g:
        thewriter = writer(g)  
        thewriter.writerow(column_names)       
        thewriter.writerows(info_list)

0 park_before_2013
0
1 park_2013
2326
2 park_2014
2376
3 park_2015
2464
4 park_2016
3572
5 park_2017
3615
6 park_2018
3639
7 park_2019
3674
8 park_2020
3693
9 park_2021
3933


In [88]:
# Features of Interest by year and locations Dataset preprocessing 
for j in range(len(station_by_year_list)):
    info_list = []
    print(j, station_by_year_name[j])

    # Name the csv file
    csv_name =  '../../data/curated/features_of_interst/train_station/' + str(station_by_year_name[j]) + '.csv'

    for k in range(j):
        resp = requests.get(url=station_by_year_list[k])

        content = json.loads(resp.text)

        # Get request for each property ID and extract attributes
        for i in content['features']:
            place_name = i['properties']['PLACE_NAME']
            longitude = i['geometry']['coordinates'][0]
            latitude = i['geometry']['coordinates'][1]

            info = [place_name, latitude, longitude]  
            info_list.append(info)

    print(len(info_list))

    with open(csv_name, 'a', newline='') as g:
        thewriter = writer(g)     
        thewriter.writerow(column_names)    
        thewriter.writerows(info_list)

0 station_before_2013
0
1 station_2013
864
2 station_2014
866
3 station_2015
868
4 station_2016
869
5 station_2017
870
6 station_2018
870
7 station_2019
878
8 station_2020
879
9 station_2021
880


In [62]:
import geopandas as gpd
import matplotlib.pyplot as plt

filename = '../../data/raw/ASGS_2021_Main_Structure_GDA2020.gpkg'
gdf = gpd.read_file(filename)


# Extract the data only for Victoria
gdf_vic_only = gdf[gdf['STATE_NAME_2021'] == 'Victoria']

# Get the size of the image right 
plt.figure(figsize=(50, 40))
# Plot Victoria
gdf_vic_only.plot(column = 'SA2_CODE_2021')

plt.savefig("Victoria_SA2.png", dpi = 800)

print(gdf_vic_only.head())

DriverError: ../../data/raw/ASGS_2021_Main_Structure_GDA2020.gpkg: No such file or directory